In [8]:
ANNOTATION_DIR = '/root/Dataset/COSMOS/cosmos_anns_acm/acm_anns/'
CONTEXT_DIR = '/root/Dataset/COSMOS/context/Context_EL/'

In [9]:
import torch
from QA import answer

import json
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

from os.path import join, isfile
from os import listdir
from itertools import groupby
import re
import numpy as np
import random

In [10]:
def get_image_id(filename: str) -> int:
    return int(re.search('([0-9]+)_[0-9]+\.txt', filename).group(1))

context_files = [join(CONTEXT_DIR, file) for file in listdir(CONTEXT_DIR) if isfile(join(CONTEXT_DIR, file))]
context_files = sorted(context_files, key=get_image_id)

context_files_grouped_by_image_id = {key: list(val) for key, val in groupby(context_files, key = get_image_id)}

test_data = pd.read_json(join(ANNOTATION_DIR, 'test_data.json'), lines=True)

In [11]:
test_data['context_filename'] = pd.DataFrame([context_files_grouped_by_image_id]).T
test_data['context_filename'] = [ [] if x is np.NaN else x for x in test_data['context_filename'] ]

In [12]:
def process(row):
    context_filename = row.context_filename
    answers = []
    for filename in context_filename:
        with open(filename, 'r', encoding='utf8') as ifile:
            content = json.load(ifile)
        context = content['context']
        if context == '':
            continue
        context = ' '.join(context.split('\n'))
        answers.append(answer(context, row.caption1, row.caption2))
    return answers

In [13]:
test_data['answers'] = test_data.progress_apply(process, axis=1)

  0%|          | 0/1000 [00:00<?, ?it/s]/root/miniconda3/envs/NLP/lib/python3.10/site-packages/transformers/pipelines/base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 1000/1000 [02:09<00:00,  7.74it/s]


In [14]:
test_data.to_csv('./df_answer_EL.csv')